# Component Catalog

> A centralized registry of all Cogitarelink tools organized by semantic layer.

This module provides a single dictionary that defines all available tools in the system, their documentation, and their layer in the semantic web stack. It serves as the single source of truth for tool discovery and documentation.

In [ ]:
#| default_exp components

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

# Component registry organized by semantic layer:
# 1. Context - Tools for handling JSON-LD contexts and namespaces
# 2. Ontology - Tools for fetching and exploring ontologies/vocabularies
# 3. Rules - Tools for validation and rules processing
# 4. Instances - Tools for data instance management
# 5. Verification - Tools for verification and signatures

COMPONENTS = {
    # ===== Basic Echo Tool (for testing) =====
    "Utils": {
        "layer": "Utility",
        "tool": "EchoMessage",
        "doc": "Simply echoes the input message back.",
        "calls": "load_module_source(module_name:str, full_name:bool=False) -> str",
        "module": "cogitarelink.utils"
    },
    
    # ===== Context Layer =====
    "ContextProcessor": {
        "layer": "Context",
        "tool": "LoadContext",
        "doc": "Loads and processes JSON-LD contexts.",
        "calls": "compact(doc:dict, ctx:dict) -> dict",
        "module": "cogitarelink.core.context"
    },
    
    # ===== Ontology Layer =====
    "VocabRegistry": {
        "layer": "Ontology",
        "tool": "FetchOntology",
        "doc": "Accesses the vocabulary registry.",
        "calls": "resolve(uri:str) -> dict",
        "module": "cogitarelink.vocab.registry"
    },
    
    # ===== Rules Layer =====
    "ValidateEntity": {
        "layer": "Rules",
        "tool": "ValidateEntity",
        "doc": "Validates an Entity against SHACL shapes.",
        "calls": "validate_entity(target:str, shapes_graph:str) -> bool",
        "module": "cogitarelink.verify.validator"
    },
    
    # ===== Instances Layer =====
    "GraphManager": {
        "layer": "Instances",
        "tool": "GraphManager",
        "doc": "Manages RDF graphs and triples.",
        "calls": "query(q:str) -> dict",
        "module": "cogitarelink.core.graph"
    },
    
    # ===== Verification Layer =====
    "Signer": {
        "layer": "Verification",
        "tool": "VerifySignature",
        "doc": "Verifies a digital signature on a named graph.",
        "calls": "verify(graph_id:str, signature:str) -> bool",
        "module": "cogitarelink.verify.signer"
    },
    
    # ===== Memory Tools =====
    "AddReflection": {
        "layer":"Utility",
        "tool":"AddReflection",
        "doc":"Persist a reflection into semantic memory",
        "calls":"add(text:str, tags:list=None)->str",
        "module":"cogitarelink_dspy.memory"
    },
    "RecallReflection": {
        "layer":"Utility",
        "tool":"RecallReflection",
        "doc":"Retrieve recent reflection notes",
        "calls":"retrieve(limit:int, tag_filter:str=None)->list",
        "module":"cogitarelink_dspy.memory"
    },
    "ReflectionPrompt": {
        "layer":"Utility",
        "tool":"ReflectionPrompt",
        "doc":"Format recent notes for prompt injection",
        "calls":"as_prompt(limit:int)->str",
        "module":"cogitarelink_dspy.memory"
    }
}

In [ ]:
#| export
def get_tools_by_layer(layer, registry=COMPONENTS):
    """Return all tool definitions for a specific layer.
    
    Args:
        layer (str): The layer name to filter by
        registry (dict, optional): The component registry to use. Defaults to COMPONENTS.
        
    Returns:
        dict: Dictionary of component name to metadata for the specified layer
    """
    return {name: meta for name, meta in registry.items() 
            if meta['layer'] == layer}

def list_layers(registry=COMPONENTS):
    """Return all unique layers in the component registry.
    
    Args:
        registry (dict, optional): The component registry to use. Defaults to COMPONENTS.
        
    Returns:
        list: Sorted list of layer names
    """
    return sorted(list(set(meta['layer'] for meta in registry.values())))

In [ ]:
# Test our component registry and helper functions
all_layers = list_layers()
print(f"Discovered layers: {all_layers}")
assert "Context" in all_layers
assert "Rules" in all_layers
assert "Instances" in all_layers

# Test getting tools by layer
utility_tools = get_tools_by_layer("Utility")
assert "Echo" in utility_tools
assert utility_tools["Echo"]["tool"] == "EchoMessage"

# Display a sample of documentation
for name, meta in list(COMPONENTS.items())[:2]:
    print(f"Tool: {meta['tool']} [Layer: {meta['layer']}]")
    print(f"Doc: {meta['doc']}")
    print(f"Calls: {meta['calls']}")
    print("-" * 40)

In [ ]:
#| export
def validate_component_registry(registry=COMPONENTS):
    """Validate that all entries in the component registry have required fields."""
    # Module field is optional; only layer, tool, doc, and calls are required
    required_fields = ['layer', 'tool', 'doc', 'calls']
    errors = []
    
    for name, meta in registry.items():
        # Check required fields
        for field in required_fields:
            if field not in meta:
                errors.append(f"Component {name} is missing required field '{field}'")
        
        # Validate tool name is a valid Python identifier
        if 'tool' in meta and not meta['tool'].isidentifier():
            errors.append(f"Component {name} has invalid tool name '{meta['tool']}', must be a valid Python identifier")
        
        # Validate calls format
        if 'calls' in meta:
            try:
                # Attempt to parse the signature
                call_sig = meta['calls']
                if '(' not in call_sig or ')' not in call_sig:
                    errors.append(f"Component {name} has malformed calls '{call_sig}', expected format 'method(param:type, ...)'")
            except Exception as e:
                errors.append(f"Component {name} has invalid calls string: {str(e)}")
                
    # Check for duplicates
    tools = {}
    for name, meta in registry.items():
        if 'tool' in meta:
            tool_name = meta['tool']
            if tool_name in tools:
                errors.append(f"Duplicate tool name '{tool_name}' in components '{name}' and '{tools[tool_name]}'")
            else:
                tools[tool_name] = name
                
    return errors

In [ ]:
# Test the validation function
errors = validate_component_registry()
assert len(errors) == 0, f"Found errors in component registry: {errors}"

# Test adding invalid components
invalid_registry = COMPONENTS.copy()
invalid_registry["BadComponent"] = {
    "layer": "Utility",
    "tool": "Invalid Tool Name",  # Contains spaces, invalid identifier
    "doc": "This is a bad component."
    # Missing calls and module
}

errors = validate_component_registry(invalid_registry)
print(f"Validation found {len(errors)} errors in the invalid registry:")
for error in errors:
    print(f"- {error}")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()